In [1]:
#%matplotlib nbagg
#%pylab

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
#from collections import Counter

In [3]:
events = pd.read_csv('events_up_to_01062018.csv')
labels = pd.read_csv('labels_training_set.csv')

c:\users\bangho\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
events.timestamp = pd.to_datetime(events.timestamp)

In [5]:
whole_data = events.merge(labels, on='person', how='left')

In [6]:
len(whole_data.index)

2341681

In [7]:
whole_data.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,label
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Aggregamos la feature diff_days

def get_days_from_last_event_specific(dataframe, type_event):
    
    if type_event != 'none':
        
        dataframe = dataframe[dataframe['event']==type_event]
    
    g1 = dataframe.groupby(["person"])['timestamp'].max()
    
    df_time_last_event = pd.DataFrame(g1)
    
    df_time_last_event = df_time_last_event.reset_index()
    
    df_time_last_event['diff_time'] = pd.to_datetime('2018/06/01') - df_time_last_event['timestamp']
    
    name_column = 'diff_time_days' + type_event
    
    df_time_last_event[name_column] = round(df_time_last_event['diff_time']/np.timedelta64(1, 'D'),1)
    
    return df_time_last_event

In [9]:
df_last_time = get_days_from_last_event_specific(whole_data, 'none')

In [10]:
df_last_time.head(3)

,person,timestamp,diff_time,diff_time_daysnone
0,0008ed71,2018-05-17 16:28:37,14 days 07:31:23,14.3
1,00091926,2018-05-31 19:52:03,0 days 04:07:57,0.2
2,00091a7a,2018-03-26 14:56:58,66 days 09:03:02,66.4


In [11]:
whole_data = pd.merge(whole_data, df_last_time[['person', 'diff_time_daysnone']], on='person', how='left')

In [12]:
def get_days_from_last_event_specific_min(dataframe, type_event):
    
    if type_event != 'none':
        
        dataframe = dataframe[dataframe['event']==type_event]
    
    g1 = dataframe.groupby(["person"])['timestamp'].min()
    
    df_time_last_event = pd.DataFrame(g1)
    
    df_time_last_event = df_time_last_event.reset_index()
    
    df_time_last_event['diff_time'] = pd.to_datetime('2018/06/01') - df_time_last_event['timestamp']
    
    name_column = 'diff_time_days_min' + type_event
    
    df_time_last_event[name_column] = round(df_time_last_event['diff_time']/np.timedelta64(1, 'D'),1)
    
    return df_time_last_event

In [13]:
df_last_time_min = get_days_from_last_event_specific_min(whole_data, 'none')

In [14]:
df_last_time_min.head()

,person,timestamp,diff_time,diff_time_days_minnone
0,0008ed71,2018-05-17 12:27:47,14 days 11:32:13,14.5
1,00091926,2018-05-03 22:08:29,28 days 01:51:31,28.1
2,00091a7a,2018-03-26 14:51:11,66 days 09:08:49,66.4
3,000ba417,2018-05-17 11:11:45,14 days 12:48:15,14.5
4,000c79fe,2018-05-29 00:27:47,2 days 23:32:13,3.0


In [15]:
whole_data = pd.merge(whole_data, df_last_time_min[['person', 'diff_time_days_minnone']], on='person', how='left')

In [16]:
# Agregamos cantidad de dias desde el ultimo checkout

df_last_time_checkout = get_days_from_last_event_specific(whole_data, 'checkout')

In [17]:
whole_data = pd.merge(whole_data, df_last_time_checkout[['person', 'diff_time_dayscheckout']], on='person', how='left')

In [18]:
whole_data['diff_time_dayscheckout'] = whole_data['diff_time_dayscheckout'].fillna(-1)

In [19]:
# Agregamos cantidad de dias desde la ultima conversion

df_last_time_conversion = get_days_from_last_event_specific(whole_data, 'conversion')

whole_data = pd.merge(whole_data, df_last_time_conversion[['person', 'diff_time_daysconversion']], on='person', how='left')

In [20]:
whole_data['diff_time_daysconversion'] = whole_data['diff_time_daysconversion'].fillna(-1)

In [21]:
# brand listing

df_last_time_brand_listing = get_days_from_last_event_specific(whole_data, 'brand listing')

whole_data = pd.merge(whole_data, df_last_time_brand_listing[['person', 'diff_time_daysbrand listing']], on='person', how='left')

In [22]:
whole_data['diff_time_daysbrand listing'] = whole_data['diff_time_daysbrand listing'].fillna(-1)

In [23]:
# search

df_last_time_search = get_days_from_last_event_specific(whole_data, 'search engine hit')

whole_data = pd.merge(whole_data, df_last_time_search[['person', 'diff_time_dayssearch engine hit']], on='person', how='left')

In [24]:
whole_data['diff_time_dayssearch engine hit'] = whole_data['diff_time_dayssearch engine hit'].fillna(-1)

In [25]:
# Agregamos diferencia dia primer evento y ultimo evento

whole_data['diff_time_start_end'] = whole_data['diff_time_days_minnone'] - whole_data['diff_time_daysnone']

In [26]:
# Agregamos la feature de los productos que se buscaron cuantas veces fue buscado el mas buscado

In [27]:
# Agregamos la feature cantidad de veces que le usuario observo el celular que mas vio

df_view_product = whole_data[whole_data['event']=='viewed product']

g_view_product = df_view_product.groupby(["person", 'model'])['model'].count()

df_view_product_per_user = pd.DataFrame(g_view_product)

In [28]:
df_view_product_per_user = pd.DataFrame(g_view_product)

df_view_product_per_user = df_view_product_per_user.rename(columns={'model': 'number_visits_model'})

In [29]:
df_view_product_per_user = df_view_product_per_user.reset_index()

In [30]:
df_view_product_per_user.head(3)

,person,model,number_visits_model
0,00091926,LG G4 H815P,1
1,00091926,LG G5 SE,1
2,00091926,Lenovo Vibe K5,1


In [31]:
# Agregamos la feature de si el telefono mas visto es o no un Iphone

def is_word_in_string(word, my_pandas_series):
    
    array_results = []
    
    for item in my_pandas_series:
        
        if word in item: 
            array_results.append(1)
        else:
            array_results.append(0)
            
    return array_results

In [32]:
df_view_product_per_user['most_visit_product_iphone'] = is_word_in_string('iPhone', df_view_product_per_user['model'])

In [33]:
whole_data = pd.merge(whole_data, df_view_product_per_user[['person', 'number_visits_model', 'most_visit_product_iphone']], on='person', how='left')

In [34]:
# Completamos con 0 los nans

whole_data['number_visits_model'] = whole_data['number_visits_model'].fillna(0)

whole_data['most_visit_product_iphone'] = whole_data['most_visit_product_iphone'].fillna(0)

In [35]:
# Agregamos la feature tiempo promedio entre eventos

df_unique_users = pd.DataFrame(whole_data['person'].unique())

df_unique_users = df_unique_users.rename(columns={0: 'person'})

In [36]:
df_logs_users = whole_data.sort_values(['person', 'timestamp'], ascending=[True, True])

df_logs_users['time_diff'] = df_logs_users.groupby('person')['timestamp'].diff()

df_logs_users['time_diff'] = df_logs_users['time_diff'].dt.total_seconds()/60/60/24

In [37]:
df_logs_users = df_logs_users[['person','timestamp', 'time_diff']]

In [38]:
df_logs_users['id'] = df_logs_users.index

In [39]:
import featuretools as ft

es = ft.EntitySet('app_data')

es.entity_from_dataframe(
    entity_id='users', # define entity id
    dataframe=df_unique_users, # select underlying data
    index='person', # define unique index column
    #time_index="timestamp",
)

es.entity_from_dataframe(
    entity_id='users_logs', # define entity id
    dataframe=df_logs_users, # select underlying data
    index='id', # define unique index column
    time_index="timestamp",
)

users_to_users_logs = ft.Relationship(
    es['users']['person'],
    es['users_logs']['person']
)

# ===> Agregamos la relationship al entity set

es = es.add_relationship(users_to_users_logs)

feature_matrix, features = ft.dfs(entityset=es, 
                                  target_entity="users", 
                                  agg_primitives=["count", "mean"], 
                                  trans_primitives=[],
                                  features_only=False)
print(feature_matrix.head(3))

          COUNT(users_logs)  MEAN(users_logs.time_diff)
person                                                 
0008ed71                  6                    0.033449
00091926              16128                    0.001730
00091a7a                 30                    0.000138


In [40]:
df_time_between_events_per_user = pd.DataFrame(feature_matrix)

In [41]:
df_time_between_events_per_user = df_time_between_events_per_user.reset_index()

In [42]:
df_time_between_events_per_user = df_time_between_events_per_user.rename(columns={0: 'person', 'MEAN(users_logs.time_diff)': 'avg_time_events'})

In [43]:
whole_data = pd.merge(whole_data, df_time_between_events_per_user[['person', 'avg_time_events']], on='person', how='left')

In [44]:
whole_data['avg_time_events'] = whole_data['avg_time_events'].fillna(-1)

In [45]:
df_time_between_events_per_user.head(3)

,person,COUNT(users_logs),avg_time_events
0,0008ed71,6,0.033449
1,00091926,16128,0.001730
2,00091a7a,30,0.000138


In [46]:
# Feature extraction

# (las lineas comentadas son features que no aumentaban la capacidad predictiva del modelo)

def get_person_features(person_data):
    event_types = person_data.event.value_counts().to_dict()
    device_types = person_data.device_type.value_counts().to_dict()
    os_types = person_data.operating_system_version.dropna().map(lambda x: x.split()[0]).value_counts().to_dict()
    
    features = {}
    features["n_events"] = len(person_data)
    features['diff_time_day'] = person_data['diff_time_daysnone'].mean()
    
    features['diff_time_brand'] = person_data['diff_time_daysbrand listing'].mean()
    features['diff_time_search'] = person_data['diff_time_dayssearch engine hit'].mean()
    
    features['diff_time_dayscheckout'] = person_data['diff_time_dayscheckout'].mean()
    features['number_visits_model'] = person_data['number_visits_model'].mean()
    features['avg_time_events'] = person_data['avg_time_events'].mean()
   
    #features['diff_time_daysconversion'] = person_data['diff_time_daysconversion'].mean()
    #features['most_visit_product_iphone'] = person_data['most_visit_product_iphone'].mean()
    #features['diff_time_start_end'] = person_data['diff_time_start_end'].mean()
    features.update(event_types)
    features.update(device_types)
    features.update(os_types)
    
    return features

In [47]:
# Sacamos los usuarios que no tienen labels 

In [48]:
whole_data_train = whole_data[~pd.isna(whole_data.label)] 

In [49]:
labels_dict = labels.set_index('person').label.to_dict()
whole_dataset = []
for person, person_indices in tqdm(whole_data_train.groupby('person').groups.items()):
    person_data = whole_data_train.loc[person_indices]
    person_data.pop('label') 
 
    whole_dataset.append({'person': person,'y': labels_dict[person],'x': get_person_features(person_data)})

100%|███████████████████████████████████████████████████████████████████████████| 19414/19414 [01:48<00:00, 179.45it/s]


In [50]:
whole_dataset

[{'person': '0008ed71',
  'y': 0,
  'x': {'n_events': 6,
   'diff_time_day': 14.299999999999999,
   'diff_time_brand': -1.0,
   'diff_time_search': -1.0,
   'diff_time_dayscheckout': 14.299999999999999,
   'number_visits_model': 0.0,
   'avg_time_events': 0.033449074074074076,
   'checkout': 3,
   'visited site': 2,
   'generic listing': 1,
   'Computer': 2,
   'Windows': 2}},
 {'person': '000c79fe',
  'y': 0,
  'x': {'n_events': 17,
   'diff_time_day': 3.0,
   'diff_time_brand': -1.0,
   'diff_time_search': 3.0,
   'diff_time_dayscheckout': 3.0,
   'number_visits_model': 3.0,
   'avg_time_events': 0.0004484953703703704,
   'searched products': 9,
   'viewed product': 3,
   'ad campaign hit': 1,
   'generic listing': 1,
   'checkout': 1,
   'visited site': 1,
   'search engine hit': 1,
   'Smartphone': 1,
   'Android': 1}},
 {'person': '001802e4',
  'y': 0,
  'x': {'n_events': 38,
   'diff_time_day': 0.3000000000000001,
   'diff_time_brand': -1.0,
   'diff_time_search': -1.0,
   'diff_

In [51]:
# Trainig/development/test

from random import Random

n_rows = len(whole_dataset)
tr_rows = int(n_rows * 0.8)
dev_rows = int((n_rows - tr_rows) / 2)

whole_dataset.sort(key=lambda x: x["person"])
Random(44).shuffle(whole_dataset) 
tr = whole_dataset[:tr_rows] 
dev = whole_dataset[tr_rows:tr_rows + dev_rows] 
test = whole_dataset[tr_rows + dev_rows:] 

print(len(tr), len(dev), len(test))

15531 1941 1942


In [52]:
# Separo cada dataset en X e y. 

def project_set(dataset):
    X = [row['x'] for row in dataset]
    y = [row['y'] for row in dataset]
    return X, y

tr_X, tr_y = project_set(tr)
dev_X, dev_y = project_set(dev)
test_X, test_y = project_set(test)

In [53]:
# Armo el vectorizador con training 
from sklearn.feature_extraction import DictVectorizer 
vectorizer = DictVectorizer(sparse=False).fit(tr_X)

In [54]:
# Llevo todo a vectores
tr_X = vectorizer.transform(tr_X)
dev_X = vectorizer.transform(dev_X)

In [55]:
test_X = vectorizer.transform(test_X)

In [56]:
# Regresión logística 
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression().fit(tr_X, tr_y)

c:\users\bangho\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bangho\python\python37\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [57]:
# Mido los resultados
from sklearn.metrics import roc_auc_score

print("tr roc auc: {}".format(roc_auc_score(tr_y, lr.predict_proba(tr_X)[:,1])))
print("dev roc auc: {}".format(roc_auc_score(dev_y, lr.predict_proba(dev_X)[:,1])))

tr roc auc: 0.45669950697411615
dev roc auc: 0.4274211199299611


In [58]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB().fit(tr_X, tr_y)

print("tr percision: {}".format(roc_auc_score(tr_y, gnb.predict_proba(tr_X)[:,1])))
print("dev percision: {}".format(roc_auc_score(dev_y, gnb.predict_proba(dev_X)[:,1])))

tr percision: 0.7270250247647798
dev percision: 0.7160029490029837


In [59]:
# Entreno nearest neighbors
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier().fit(tr_X, tr_y)
print("tr percision: {}".format(roc_auc_score(tr_y, knn.predict_proba(tr_X)[:,1])))
print("dev percision: {}".format(roc_auc_score(dev_y, knn.predict_proba(dev_X)[:,1])))

tr percision: 0.942087857064621
dev percision: 0.5830731836560725


In [60]:
# Entreno Gradient Boosting
#from sklearn.model_selection import GridSearchCV
#from sklearn.ensemble import GradientBoostingClassifier

#gbc_grid_params = {'learning_rate': [0.2,0.1,0.05],
#              'max_depth': [2,3,5,8],
#              
#                  'n_estimators': [10,20,35,50],
#              'max_features': [5, 10, 20] 
#              }

#gbc = GradientBoostingClassifier(random_state=4)

#gbc_grid = GridSearchCV(gbc,gbc_grid_params,cv=2,verbose = 3,n_jobs=10);

#gbc_grid.fit(tr_X, tr_y)

#best_params_gbc = gbc_grid.best_params_

#gbc = GradientBoostingClassifier().fit(tr_X, tr_y)

#print("tr roc auc: {}".format(roc_auc_score(tr_y, gbc.predict_proba(tr_X)[:,1])))
#print("dev roc auc: {}".format(roc_auc_score(dev_y, gbc.predict_proba(dev_X)[:,1])))

In [61]:
#model_gbc = GradientBoostingClassifier( learning_rate=best_params_gbc.get('learning_rate'), 
#                                   max_depth=best_params_gbc.get('max_depth'),
#                                  
#                     n_estimators=best_params_gbc.get('n_estimators'))
#
#model_gbc.fit(tr_X, tr_y)

In [62]:
#print("tr roc auc: {}".format(roc_auc_score(tr_y, model_gbc.predict_proba(tr_X)[:,1])))
#print("dev roc auc: {}".format(roc_auc_score(dev_y, model_gbc.predict_proba(dev_X)[:,1])))

In [63]:
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBClassifier

gb_grid_params = {'learning_rate': [0.2,0.1,0.05],
              'max_depth': [1,2,3,5,],
              'gamma':[0.01,0.03, 0.05,0.1],
                  'n_estimators': [20,35,50,100]
              #'max_features': [1.0, 0.3, 0.1] 
              }

xgb = XGBClassifier(random_state=3)

xgb_grid = GridSearchCV(xgb,gb_grid_params,cv=2,verbose = 3,n_jobs=10);

xgb_grid.fit(tr_X, tr_y)

best_params = xgb_grid.best_params_

print(xgb_grid.best_estimator_, xgb_grid.best_score_, xgb_grid.best_params_)

Fitting 2 folds for each of 192 candidates, totalling 384 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    4.3s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:   21.4s
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:   53.0s
[Parallel(n_jobs=10)]: Done 384 out of 384 | elapsed:  1.3min finished


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.01, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=3,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1) 0.9495847015646127 {'gamma': 0.01, 'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 50}


In [64]:
model = XGBClassifier( learning_rate=best_params.get('learning_rate'), 
                                   max_depth=best_params.get('max_depth'),
                                  gamma=best_params.get('gamma'),
                     n_estimators=best_params.get('n_estimators'))

model.fit(tr_X, tr_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.01, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [ ]:
y_pred_proba = model.predict_proba(dev_X)

In [ ]:
roc_auc_score(dev_y, y_pred_proba[:,1])

0.8984984275823936

In [ ]:
from xgboost import plot_importance
from matplotlib import pyplot

plot_importance(model)
pyplot.show()

<Figure size 640x480 with 1 Axes>

In [ ]:
dev_X

array([[2.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 6.0000e+00,
        2.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 4.0000e+00,
        4.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 6.8200e+02, 3.4348e+04,
        1.6740e+03],
       ...,
       [6.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 1.2000e+01,
        6.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 6.0000e+00,
        2.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 2.4000e+01,
        1.2000e+01]])

In [ ]:
y_pred_proba[:,1]

array([0.01208308, 0.01955785, 0.0356418 , ..., 0.0128357 , 0.01292031,
       0.19532779], dtype=float32)

In [ ]:
# Predecimos una label para los usuarios sin labels

whole_data_submision = whole_data[pd.isna(whole_data.label)]

In [ ]:
#labels_dict = labels.set_index('person').label.to_dict()
whole_dataset_submision = []
for person, person_indices in tqdm(whole_data_submision.groupby('person').groups.items()):
    person_data = whole_data_submision.loc[person_indices]
    #person_data.pop('label') 
 
    whole_dataset_submision.append({'person': person,'y': 0,'x': get_person_features(person_data)})

In [ ]:
whole_dataset_submision

In [ ]:
array_user_ids = []

for user in whole_dataset_submision:
    array_user_ids.append(user['person'])

In [ ]:
submision_X, submision_y = project_set(whole_dataset_submision)

In [ ]:
submision_X = vectorizer.transform(submision_X)

In [ ]:
y_pred_proba_sub = model.predict_proba(submision_X)

In [ ]:
df_final_co = pd.DataFrame(array_user_ids)

df_final_co = df_final_co.rename(columns={0: 'person'})

In [ ]:
df_final_co['label'] = y_pred_proba_sub[:,1]

In [ ]:
df_final_co.head()

In [ ]:
df_final_co.to_csv('sub_tom_co.csv', index=False)